In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import datetime

from typing import List, Dict

# Stock Prices EDA

In [5]:
START_DATE = dt.datetime(2020,11,12)
END_DATE = dt.datetime(2021,2,12)

In [67]:
class SmigStock:

    DATE_FORMAT = "%Y-%m-%d"

    @property
    def tickers(self):
        return self.tickers

    @tickers.setter
    def tickers(self, tickers: List[str]):
        if tickers and isinstance(tickers, list):
            self.tickers = tickers

    def add_ticker(self, ticker: str) -> None:
        self.tickers.append(ticker)

    @property
    def start_date(self) -> "datetime.datetime":
        return self.start_date

    @start_date.setter
    def start_date(self, start_date: str) -> None:
        if not isinstance(end_date, str):
            return
        try:
            self.start_date = datetime.datetime.strptime(start_date, SmigStock.DATE_FORMAT)
        except Exception as e:
            print(str(e))
            raise

    @property
    def end_date(self) -> "datetime.datetime":
        return self.end_date

    @start_date.setter
    def end_date(self, end_date: str) -> None:
        if not isinstance(end_date, str):
            return
        try:
            self.end_date = datetime.datetime.strptime(end_date, SmigStock.DATE_FORMAT)
        except Exception as e:
            print(str(e))
            raise

    def __init__(self, tickers: List[str], start_date: str = None, end_date: str = None) -> None:
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.data = {}

    def get_prices(self) -> None:
        for ticker in self.tickers:
            df = web.DataReader(ticker, 'yahoo', start_date, end_date)
            df.reset_index(inplace=True)
            df = df[['Date', 'Adj Close']]
            df.columns = ['ds', 'y']
            self.data[ticker] = df

In [3]:
def get_stock_data(ticker, start_date, end_date):
    df = web.DataReader(ticker, 'yahoo', start_date, end_date)
    df.reset_index(inplace=True)
    df = df[['Date', 'Adj Close']]
    df.columns = ['ds', 'y']
    return df

In [46]:
df = get_stock_data('TSLA', START_DATE, END_DATE)
df.set_index('ds', inplace=True)
df['daily_ret'] = df.diff(axis=0)
df['daily_percent'] = df['daily_ret'] / df['y'].shift()
df.head()

y  daily_ret  daily_percent
ds                                              
2020-11-12  411.760010        NaN            NaN
2020-11-13  408.500000  -3.260010      -0.007917
2020-11-16  408.089996  -0.410004      -0.001004
2020-11-17  441.609985  33.519989       0.082139
2020-11-18  486.640015  45.030029       0.101968

## Compute Sharp Ratio

Risk-free rate is ~ $0.08$ for 2021

$s_a=\frac{E[R_a - R_b]}{\sigma_a}$


In [51]:
rf = 0.05/100
df['excess'] = df.daily_percent-rf
df.head()

y  daily_ret  daily_percent    excess
ds                                                        
2020-11-12  411.760010        NaN            NaN       NaN
2020-11-13  408.500000  -3.260010      -0.007917 -0.008417
2020-11-16  408.089996  -0.410004      -0.001004 -0.001504
2020-11-17  441.609985  33.519989       0.082139  0.081639
2020-11-18  486.640015  45.030029       0.101968  0.101468

In [61]:
s = (df.shape[0]/np.sqrt(252))*df.daily_percent.mean()/df.daily_percent.std()
print("Sharp Ratio for Tesla = %s" % s)

Sharp Ratio for Tesla = 1.2128319245357249
